In [2]:

from os import environ
from redis import Redis
from time import sleep

stream_key = environ.get("STREAM", "STREAM")
producer = environ.get("PRODUCER", "project-a")


def connect_to_redis():
    hostname = environ.get("REDIS_HOSTNAME", "redis-service") 
    port = environ.get("REDIS_PORT", 6379)

    return Redis(hostname, port, retry_on_timeout=True)


def send_event(redis_connection, reference):
    count = 0

    try:
        # TODO cloud events
        # un-map the "data" wrt app package parameters
        data = {
            "subject": reference,
            "producer": producer,
            "href": reference,
        }
        resp = redis_connection.xadd(stream_key, data)
        print(resp)
        count += 1

    except ConnectionError as e:
        print(f"ERROR REDIS CONNECTION: {e}")



In [3]:

connection = connect_to_redis()


In [6]:
references = [
    "https://earth-search.aws.element84.com/v1/collections/sentinel-2-l2a/items/S2B_10TFK_20211230_0_L2A",
    "https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2D_10TFK_20210708_0_L2A",
    "https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2A_10TFK_20210708_0_L2A",
    "https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2B_10TFK_20210713_0_L2A",
    "https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2A_10TFK_20210718_0_L2A",
    "https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2A_10TFK_20220524_0_L2A",
    "https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2D_10TFK_20220524_0_L2A",
    "https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2A_10TFK_20220514_0_L2A",
    "https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2A_10TFK_20220504_0_L2A"
]

for reference in references:
    send_event(connection, reference)
    break
    sleep(1)


b'1729601939525-0'


In [12]:
import pystac_client
from datetime import datetime, timedelta
from time import sleep
# Define the STAC API URL
stac_api_url = "https://earth-search.aws.element84.com/v1"

# Create a client instance
client = pystac_client.Client.open(stac_api_url)

# Array of event dates (ISO 8601 format)
start_date = datetime.fromisoformat("2021-07-04T00:00:00Z".replace("Z", "+00:00"))

# Number of dates and interval in days
num_dates = 60
interval_days = 5

dates = [date.strftime("%Y-%m-%dT%H:%M:%SZ") for date in [start_date + timedelta(days=i * interval_days) for i in range(num_dates)]]

# Loop over each event date
for event_date_str in event_dates:
    # Parse the event date
    event_date = datetime.fromisoformat(event_date_str.replace("Z", "+00:00"))
    
    # Query four times with 6-hour steps
    for i in range(4):
        # Calculate the start and end times for this step
        start_time = event_date - timedelta(minutes=(6 * 60 * (i + 1)))
        end_time = event_date - timedelta(minutes=(6 * 60 * i) + 1)
        
        
        # Format the time range in ISO 8601 format
        time_of_interest = [start_time.strftime('%Y-%m-%dT%H:%M:%SZ') , end_time.strftime('%Y-%m-%dT%H:%M:%SZ')]
        
        print(time_of_interest)

        # Perform the search query
        search = client.search(
            collections=["sentinel-2-l2a"],
            bbox="-121.399,39.834,-120.74,40.472",
            datetime=time_of_interest,
            max_items=50  # Limit the number of returned items
        )
        
        # Fetch the search results
        items = list(search.items())
        
        # Display some basic information about the found items
        print(f"Results for event date: {event_date_str}, Interval {i+1}")
        print(f"Time of interest: {time_of_interest}")
        if items:
            for item in items:
                print(f"ID: {item.id}")
                print(f"Date: {item.datetime}")
                print(f"Assets: {list(item.assets.keys())}")
                print(f"Bounding box: {item.bbox}")
                print()
                send_event(connection, [link.href for link in item.links if link.rel in ["self"]][0])
        else:
            print("No results found.")
        print("-" * 40)
        sleep(5)


    sleep(90)
    

['2021-07-03T18:00:00Z', '2021-07-03T23:59:00Z']
Results for event date: 2021-07-04T00:00:00Z, Interval 1
Time of interest: ['2021-07-03T18:00:00Z', '2021-07-03T23:59:00Z']
No results found.
----------------------------------------
['2021-07-03T12:00:00Z', '2021-07-03T17:59:00Z']
Results for event date: 2021-07-04T00:00:00Z, Interval 2
Time of interest: ['2021-07-03T12:00:00Z', '2021-07-03T17:59:00Z']
No results found.
----------------------------------------
['2021-07-03T06:00:00Z', '2021-07-03T11:59:00Z']
Results for event date: 2021-07-04T00:00:00Z, Interval 3
Time of interest: ['2021-07-03T06:00:00Z', '2021-07-03T11:59:00Z']
No results found.
----------------------------------------
['2021-07-03T00:00:00Z', '2021-07-03T05:59:00Z']
Results for event date: 2021-07-04T00:00:00Z, Interval 4
Time of interest: ['2021-07-03T00:00:00Z', '2021-07-03T05:59:00Z']
No results found.
----------------------------------------
['2021-07-08T18:00:00Z', '2021-07-08T23:59:00Z']
Results for event date: